# Preliminary work

In [1]:
# importing libraries

import pandas as pd
import numpy as np
import missingno as msno
import pandas_profiling

import plotly.express
import seaborn as sns
import matplotlib.pyplot as plt

import os
from subprocess import check_output

In [2]:
print(check_output(['ls', '../pokemon-challenge']).decode('utf-8'))

Note_2.ipynb
combats.csv
note_1.ipynb
note_3.ipynb
pokemon.csv
temp-plot.html
tests.csv



# Exploratory Data Analysis



## Dtypes

In [3]:
combats = pd.read_csv('combats.csv')

In [4]:
pokemons = pd.read_csv('pokemon.csv')

In [5]:
test = pd.read_csv('tests.csv')

In [6]:
pandas_profiling.ReportData(pokemons)

AttributeError: module 'pandas_profiling' has no attribute 'ReportData'

In [ ]:
pandas_profiling.ProfileReport(pokemons)

In [7]:
pokemons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
#             800 non-null int64
Name          799 non-null object
Type 1        800 non-null object
Type 2        414 non-null object
HP            800 non-null int64
Attack        800 non-null int64
Defense       800 non-null int64
Sp. Atk       800 non-null int64
Sp. Def       800 non-null int64
Speed         800 non-null int64
Generation    800 non-null int64
Legendary     800 non-null bool
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [8]:
pokemons.describe()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.0000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,400.5000,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,231.0844,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.0000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,200.7500,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,400.5000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,600.2500,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,800.0000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [9]:
pokemons.dtypes

#              int64
Name          object
Type 1        object
Type 2        object
HP             int64
Attack         int64
Defense        int64
Sp. Atk        int64
Sp. Def        int64
Speed          int64
Generation     int64
Legendary       bool
dtype: object

In [10]:
pokemons.isna().sum()

#               0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [11]:
pokemons[pokemons['Name'].isna()] # missing pokemon

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False


Quick notes

Missing values
1. We have 386 missing values in type 2 columns due to absence of type 2. Not all pokemons have more than 1 type.

2. In name column we have one missing pokemon

3. We have 800 pokemons and 50 000 battles


# EDA in depth

## Missing values

In [12]:
pokemons[pokemons['Name'].isna()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False


In [13]:
pokemons.loc[61:63, ]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
61,62,Mankey,Fighting,NaN,40,80,35,35,45,70,1,False
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False
63,64,Growlithe,Fire,NaN,55,70,45,70,50,60,1,False


In [14]:
# Inserting missing pokemon
pokemons.loc[62, 'Name'] = 'Primeape'

In [15]:
total_wins = combats['Winner'].value_counts()

In [16]:
number_of_wins = combats.groupby('Winner').count()

In [17]:
# both methods produce the same results

In [18]:
countByFirst = combats.groupby('Second_pokemon').count()

In [19]:
countBySecond = combats.groupby('First_pokemon').count()
print("Looking at the dimensions of our dataframes")
print("Count by first winner shape: " + str(countByFirst.shape))
print("Count by second winner shape: " + str(countBySecond.shape))
print("Total Wins shape : " + str(total_wins.shape))

Looking at the dimensions of our dataframes
Count by first winner shape: (784, 2)
Count by second winner shape: (784, 2)
Total Wins shape : (783,)


In [20]:
find_losing_pokemon = np.setdiff1d(countByFirst.index.values, number_of_wins.index.values)-1

In [21]:
pokemons.loc[find_losing_pokemon]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
230,231,Shuckle,Bug,Rock,20,10,230,10,230,5,2,False


In [22]:
numberofwins = number_of_wins.sort_index()

In [23]:
numberofwins['Total Fights'] = countByFirst['Winner'] + countBySecond['Winner']

In [24]:
numberofwins['Win Percentage']= numberofwins.First_pokemon/numberofwins['Total Fights']


In [25]:
# merge the winning dataset and the original pokemon dataset

results2 = pd.merge(pokemons, numberofwins, right_index = True, left_on='#')

In [26]:
results3 = pd.merge(pokemons, numberofwins, left_on='#', right_index = True, how='left')

In [27]:
pokemons[results3['Win Percentage'].isnull()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
11,12,Blastoise,Water,NaN,79,83,100,85,105,78,1,False
32,33,Sandshrew,Ground,NaN,50,75,85,20,30,40,1,False
45,46,Wigglytuff,Normal,Fairy,140,70,45,85,50,45,1,False
65,66,Poliwag,Water,NaN,40,50,40,40,40,90,1,False
77,78,Victreebel,Grass,Poison,80,105,65,100,70,70,1,False
89,90,Magneton,Electric,Steel,50,60,95,120,70,70,1,False
143,144,Ditto,Normal,NaN,48,48,48,48,48,48,1,False
182,183,Ariados,Bug,Poison,70,90,70,60,60,40,2,False
230,231,Shuckle,Bug,Rock,20,10,230,10,230,5,2,False
235,236,Ursaring,Normal,NaN,90,130,75,75,75,55,2,False


In [28]:
# pokemons with the highest win percentage

results2.sort_values(by='Win Percentage').nlargest(10, columns='Win Percentage')

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,First_pokemon,Second_pokemon,Total Fights,Win Percentage
154,155,Mega Aerodactyl,Rock,Flying,80,135,85,70,95,150,1,False,127,127,129,0.984496
512,513,Weavile,Dark,Ice,70,120,65,45,85,125,4,False,116,116,119,0.974790
703,704,Tornadus Therian Forme,Flying,NaN,79,100,80,110,90,121,5,True,121,121,125,0.968000
19,20,Mega Beedrill,Bug,Poison,65,150,40,15,80,145,1,False,115,115,119,0.966387
153,154,Aerodactyl,Rock,Flying,80,105,65,60,75,130,1,False,136,136,141,0.964539
476,477,Mega Lopunny,Normal,Fighting,65,136,94,54,96,135,4,False,124,124,129,0.961240
726,727,Greninja,Water,Dark,72,95,67,103,71,122,6,False,122,122,127,0.960630
716,717,Meloetta Pirouette Forme,Normal,Fighting,100,128,90,77,77,128,5,False,118,118,123,0.959350
164,165,Mega Mewtwo Y,Psychic,NaN,106,150,70,194,120,140,1,True,119,119,125,0.952000
349,350,Mega Sharpedo,Water,Dark,70,140,70,110,65,105,3,False,114,114,120,0.950000


In [29]:
# pokemons with lowest win percentage rate

results2.sort_values(by='Win Percentage').nsmallest(10, columns='Win Percentage')

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,First_pokemon,Second_pokemon,Total Fights,Win Percentage
289,290,Silcoon,Bug,NaN,50,35,55,25,25,15,3,False,3,3,138,0.021739
189,190,Togepi,Fairy,NaN,35,20,65,40,65,20,2,False,3,3,122,0.024590
638,639,Solosis,Psychic,NaN,45,30,40,105,50,20,5,False,4,4,129,0.031008
236,237,Slugma,Fire,NaN,40,40,40,70,40,20,2,False,4,4,123,0.032520
576,577,Munna,Psychic,NaN,76,25,45,67,55,24,5,False,5,5,128,0.039062
188,189,Igglybuff,Normal,Fairy,90,30,15,40,20,15,2,False,5,5,115,0.043478
394,395,Wynaut,Psychic,NaN,95,23,48,23,48,23,3,False,6,6,130,0.046154
209,210,Wooper,Water,Ground,55,45,45,25,25,15,2,False,6,6,125,0.048000
291,292,Cascoon,Bug,NaN,50,35,55,25,25,15,3,False,7,7,133,0.052632
752,753,Spritzee,Fairy,NaN,78,52,60,63,65,23,6,False,8,8,133,0.060150


# Visualization with Plotly

In [30]:
# Change the default stacking

import plotly.express as px

fig = px.bar(results2, x="Type 1", y="total_bill", color='Legendary', barmode='group',
             height=400)
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary', 'First_pokemon', 'Second_pokemon', 'Total Fights', 'Win Percentage'] but received: total_bill

In [ ]:
import plotly.graph_objects as go
import plotly.offline as pyo

data = []
legendary_type = [False, True]
for pokemon in legendary_type:
    x = results2[results2['Legendary'] == pokemon]['Type 1'].value_counts().index.values
    y = results2[results2['Legendary'] == pokemon]['Type 1'].value_counts().values
    data.append(go.Bar(x=x, y=y, name=pokemon))

layout = go.Layout(title="Total pokemon by type 1",
                   xaxis=dict(title="Type 1"),
                   yaxis=dict(title="Count of legendary"),
                   legend=dict(x=1.0, y=0.5),
                   # Here annotations need to create legend title
                   annotations=[
                                dict(
                                    x=1.05,
                                    y=0.55,
                                    xref="paper",
                                    yref="paper",
                                    text="Legendary",
                                    showarrow=False
                                )],
                   barmode="group")

In [ ]:
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [ ]:
data = []
legendary_type = [False, True]
for pokemon in legendary_type:
    x = results2[results2['Legendary'] == pokemon]['Type 2'].value_counts().index.values
    y = results2[results2['Legendary'] == pokemon]['Type 2'].value_counts().values
    data.append(go.Bar(x=x, y=y, name=pokemon))

layout = go.Layout(title="Total pokemon by type 2",
                   xaxis=dict(title="Type 2"),
                   yaxis=dict(title="Count of legendary"),
                   legend=dict(x=1.0, y=0.5),
                   # Here annotations need to create legend title
                   annotations=[
                                dict(
                                    x=1.07,
                                    y=0.7,
                                    xref="paper",
                                    yref="paper",
                                    text="Legendary",
                                    showarrow=False
                                )],
                   barmode="group")
fig = go.Figure(data=data, layout=layout)
pyo.iplot(fig)

In [ ]:
# Pivot table

results2.groupby('Type 1').agg({'Win Percentage':np.mean}).sort_values('Win Percentage', ascending=False)
# results2.pivot_table(index='Type 1', values='Win Percentage', aggfunc=np.mean).sort_values('Win Percentage')

In [ ]:
#sns.distplot(results3["Win Percentage"].dropna(), bins=20)
col = ['Type 1','HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Win Percentage']
#results3.loc[:,'HP':'Speed'].corr()
sns.pairplot(results3.loc[:,col].dropna())
plt.show()

In [ ]:
g = sns.PairGrid(results3.loc[:,col], diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(sns.regplot)
g.map_diag(sns.kdeplot, lw=3)

In [ ]:
results3.loc[:,col].corr()

In [ ]:
#This function was taken from the link above 
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 50)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Pokemon Feature Correlation')
    labels=['Type 1','HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Win %']
    ax1.set_xticklabels(labels,fontsize=7)
    ax1.set_yticklabels(labels,fontsize=7)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[0.00,.05,.10,.15,.20,.25,.30,.35,.40,.45,.50,.55,.60,.65,.70,.75,.8,.85,.90,.95,1])
    plt.show()

correlation_matrix(results3.loc[:,col])

In [ ]:
sns.regplot(x="Speed", y="Win Percentage", data=results3, logistic=True).set_title("Speed vs Win Percentage")

In [ ]:
sns.lmplot(x="Speed", y="Win Percentage", data=results3, hue = 'Type 1',  logistic=True)#.set_title("Speed vs Win Percentage")

# Machine Learning


## Supervised learning


- Select data
- Preprocess data
- Transform data
- Fit data to model/predict
- Validate model
- Parameter turning

In [31]:
dataset = results2

In [32]:
# drop all missing values
dataset.dropna(axis=0, how='any')
X = dataset.iloc[:, 5:11].values
y = dataset.iloc[:, 15]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

#### Multiple Linear Regression

In [33]:
def ml_linearreg(X_train, X_test, y_train, y_test):
    # fitting multiple linear regression to the trainin set
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    print(reg.score(X_train, y_train))
    #Predicting the test set results
    y_pred = reg.predict(X_test)
    
    # Validating the results
    from sklearn.metrics import mean_absolute_error
    from math import sqrt
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = sqrt(mae)
    return f'MAE: {mae}; MSE: {mse}'

In [34]:
ml_linearreg(X_train, X_test, y_train, y_test)

0.9076142320040209


'MAE: 0.06143408809230802; MSE: 0.24785900849537024'

#### SVM

In [35]:
# Feature Scaling
def ml_svm(X_train, X_test, y_train, y_test):
    # Fitting SVR to the dataset
    from sklearn.svm import SVR
    reg = SVR(kernel='linear')
    reg.fit(X_train, y_train)
    print(reg.score(X_train, y_train))
    
    # Predict Output
    y_pred = reg.predict(X_test)
    
    from sklearn.metrics import mean_absolute_error
    from math import sqrt
    mae = mean_absolute_error(y_test, y_pred)
    return mae
ml_svm(X_train, X_test, y_train, y_test)

0.9065078337237095


0.06360099438357145

In [36]:
#feature scaling not needed

def ml_decisiontree(X_train, X_test, y_train, y_test):
    # Fitting Decision Tree Regression to the dataset
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X_train, y_train)
    print(regressor.score(X_train, y_train))

    # Predicting a new result
    y_pred = regressor.predict(X_test)

    from sklearn.metrics import mean_absolute_error
    from math import sqrt
    mae = mean_absolute_error(y_test, y_pred)
    #print("Mean Absolute Error: " + str(mae))
    return mae

ml_decisiontree(X_train, X_test, y_train, y_test)

0.9997364298486605


0.05381006518627112

#### Random Forest

In [37]:
def ml_randomforest(X_train, X_test, y_train, y_test):
    # Fitting Random Forest Regression to the dataset
    from sklearn.ensemble import RandomForestRegressor
    reg = RandomForestRegressor()
    reg.fit(X_train, y_train)
    print(reg.score(X_train, y_train))
    
    # Predicting results 
    y_pred = reg.predict(X_test)
    
    from sklearn.metrics import mean_absolute_error
    from math import sqrt
    mae = mean_absolute_error(y_test, y_pred)
   
    return mae
    
ml_randomforest(X_train, X_test, y_train, y_test)

0.9933514163022409


0.047472434612171495

In [38]:
def ml_xgboost(X_train, X_test, y_train, y_test):
    import xgboost
    # fitting XGBoost to training set
    xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7)
    xgb.fit(X_train,y_train)
    print(xgb.score(X_train, y_train))
    # Prediction
    y_pred = xgb.predict(X_test)
    #print(explained_variance_score(y_pred ,y_test))
    from sklearn.metrics import mean_absolute_error
    from math import sqrt
    mae = mean_absolute_error(y_test, y_pred)
    #print("Mean Absolute Error: " + str(mae))
    return mae

ml_xgboost(X_train, X_test, y_train, y_test)

ModuleNotFoundError: No module named 'xgboost'

In [39]:
all_stats = [ml_linearreg(X_train, X_test, y_train, y_test), ml_svm(X_train, X_test, y_train, y_test), ml_decisiontree(X_train, X_test, y_train, y_test), ml_randomforest(X_train, X_test, y_train, y_test)]
#all_stats

0.9076142320040209
0.9065078337237095
0.9997364298486605
0.993337872071722


In [ ]:
# PCA

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Applying PCA
from sklearn.decomposition import PCA
#pca = PCA(n_components = None)
pca = PCA(n_components = 3)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
# Provides a vector of the variance explained by each component
explained_variance = pca.explained_variance_ratio_
print("This is the variance explained by the principle components")
print(explained_variance)

In [46]:
PCA = [ml_linearreg(X_train, X_test, y_train, y_test), ml_svm(X_train, X_test, y_train, y_test), ml_decisiontree(X_train, X_test, y_train, y_test), ml_randomforest(X_train, X_test, y_train, y_test)]

0.44239289270218385
0.44107156028061345
0.9997364454374928
0.9313586627957581


In [49]:
# reduce the features to only speed and attack. 
dataset = results2
dataset.dropna(axis=0, how='any')
# Splitting the dataset into the Training set and Test set
X = dataset.loc[:, ['Attack','Speed']].values
y = dataset.loc[:, ['Win Percentage']].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#ml_linearreg(X_train, X_test, y_train, y_test)
#ml_svm(X_train, X_test, y_train, y_test)
#ml_decisiontree(X_train, X_test, y_train, y_test)
#ml_randomforest(X_train, X_test, y_train, y_test)
#ml_xgboost(X_train, X_test, y_train, y_test)

reduced_stats = [ml_linearreg(X_train, X_test, y_train, y_test), ml_svm(X_train, X_test, y_train, y_test), ml_decisiontree(X_train, X_test, y_train, y_test), ml_randomforest(X_train, X_test, y_train, y_test)]

0.9049921104553327


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.9040040161126808
0.9930604360712907


<ipython-input-37-881ee53b2c90>:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9885390467501574


In [50]:
#compare results from the 3 trials 
ml_results = pd.DataFrame({'All Factors': all_stats, 'Reduced Factors': reduced_stats, 'PCA': PCA})
ml_results.rename(index = {0:'Linear',1:'SVM', 2:'Decision Tree', 3:'Random Forest'})

,All Factors,Reduced Factors,PCA
Linear,MAE: 0.06143408809230802; MSE: 0.2478590084953...,MAE: 0.06281246289614097; MSE: 0.2506241466741...,MAE: 0.1585051381159789; MSE: 0.39812703766006513
SVM,0.063601,0.064739,0.157473
Decision Tree,0.0538101,0.054214,0.186182
Random Forest,0.0478419,0.0494678,0.147323
